In [2]:
%load_ext autoreload
%autoreload 2

In [20]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages')

In [21]:
from Datos import Datos
import EstrategiaParticionado
import Clasificador

import numpy as np
import random
from sklearn.preprocessing import OneHotEncoder

from tabulate import tabulate

# <font color='red'>1. Entrenamiento y clasificación con Clasificador.py</font>


## <font color='blue'> 1.1 Dataset tic-tac-toe </font>

In [22]:
dataset_ttt = Datos("../ConjuntosDatos/tic-tac-toe.data")
ndata, ncols = dataset_ttt.datos.shape # nfeat = ncols-1 
print("Numero de ejemplos de entrenamiento:", ndata, ", Numero de columnas:", ncols)

Numero de ejemplos de entrenamiento: 958 , Numero de columnas: 10


In [23]:
np.random.shuffle(dataset_ttt.datos)

NB = Clasificador.ClasificadorNaiveBayes()

### 1.1.1 Validación simple

In [24]:
strat_simple = EstrategiaParticionado.ValidacionSimple(0.8)

In [25]:
NB.validacion(strat_simple, dataset_ttt, NB)

[0.28645833333333337]

### 1.1.2 Validación simple con varias repeticiones (3 reps)

In [26]:
strat_simple_rep3 = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)

In [27]:
errores = NB.validacion(strat_simple_rep3, dataset_ttt, NB)
print("Errores:", errores)
print("Error medio:", np.mean(errores), "+-", np.std(errores))

Errores: [0.328125, 0.30208333333333337, 0.265625]
Error medio: 0.2986111111111111 +- 0.02563337336846077


### 1.1.3 Validación Cruzada (5 folds)

In [28]:
strat_cross = EstrategiaParticionado.ValidacionCruzada(k_fold=5)

In [29]:
errores = NB.validacion(strat_cross, dataset_ttt, NB)
print("Errores:", errores)
print("Error medio:", np.mean(errores), "+-", np.std(errores))

Errores: [0.2879581151832461, 0.39267015706806285, 0.25130890052356025, 0.2670157068062827, 0.32460732984293195]
Error medio: 0.30471204188481676 +- 0.05037073705049505


## <font color='blue'> 1.2 Dataset German </font>

In [30]:
dataset_ger = Datos("../ConjuntosDatos/german.data")
ndata, ncols = dataset_ger.datos.shape # nfeat = ncols-1 
print("Numero de ejemplos de entrenamiento:", ndata, ", Numero de columnas:", ncols)

Numero de ejemplos de entrenamiento: 1000 , Numero de columnas: 21


In [31]:
np.random.shuffle(dataset_ger.datos)

### 1.2.1 Validación simple

In [32]:
strat_simple_ger = EstrategiaParticionado.ValidacionSimple(0.8)
NB.validacion(strat_simple_ger, dataset_ger, NB)

[0.245]

### 1.2.2 Validación simple con varias repeticiones (3 reps)

In [33]:
strat_simple_rep3_ger = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)
errores = NB.validacion(strat_simple_rep3_ger, dataset_ger, NB)
print("Errores:", errores)
print("Error medio:", np.mean(errores), "+-", np.std(errores))

Errores: [0.22999999999999998, 0.31000000000000005, 0.21499999999999997]
Error medio: 0.25166666666666665 +- 0.04169998667732271


### 1.2.3 Validación cruzada (5 folds)

In [34]:
strat_cross_ger = EstrategiaParticionado.ValidacionCruzada(k_fold=3)
errores = NB.validacion(strat_cross_ger, dataset_ger, NB)
print("Errores:", errores)
print("Error medio:", np.mean(errores), "+-", np.std(errores))

Errores: [0.25225225225225223, 0.27627627627627627, 0.2582582582582582]
Error medio: 0.26226226226226224 +- 0.01020824727446004


## <font color='green'> 1.3 Tablas de Resultados </font>

###  1.3.1 Validación simple con número variable de repeticiones

In [35]:
def val_simple(percentage):
    max_reps = 5
    L = []
    for i in range(1, max_reps+1):
        L.append([])
        strat = EstrategiaParticionado.ValidacionSimple(percentage, nreps=i)
        errores_ttt = NB.validacion(strat, dataset_ttt, NB)
        errores_ger = NB.validacion(strat, dataset_ger, NB)
        L[i-1].append(str(i))
        L[i-1].append("%.2f +- %.3f"% (np.mean(errores_ttt), np.std(errores_ttt)))
        L[i-1].append("%.2f +- %.3f"% (np.mean(errores_ger), np.std(errores_ger)))
    print("Validación Simple con " + str(int(percentage*100)) +   "% Train\n")
    print(tabulate(L, headers = ["N_REPS", "TTT_DB", "GER_DB"], tablefmt="grid")) 

In [36]:
val_simple(0.70)

Validación Simple con 70% Train

+----------+---------------+---------------+
|   N_REPS | TTT_DB        | GER_DB        |
+==========+===============+===============+
|        1 | 0.30 +- 0.000 | 0.26 +- 0.000 |
+----------+---------------+---------------+
|        2 | 0.30 +- 0.045 | 0.23 +- 0.015 |
+----------+---------------+---------------+
|        3 | 0.27 +- 0.020 | 0.26 +- 0.023 |
+----------+---------------+---------------+
|        4 | 0.29 +- 0.017 | 0.25 +- 0.029 |
+----------+---------------+---------------+
|        5 | 0.30 +- 0.011 | 0.25 +- 0.021 |
+----------+---------------+---------------+


In [37]:
val_simple(0.75)

Validación Simple con 75% Train

+----------+---------------+---------------+
|   N_REPS | TTT_DB        | GER_DB        |
+==========+===============+===============+
|        1 | 0.30 +- 0.000 | 0.27 +- 0.000 |
+----------+---------------+---------------+
|        2 | 0.27 +- 0.004 | 0.22 +- 0.016 |
+----------+---------------+---------------+
|        3 | 0.31 +- 0.005 | 0.25 +- 0.010 |
+----------+---------------+---------------+
|        4 | 0.33 +- 0.027 | 0.25 +- 0.022 |
+----------+---------------+---------------+
|        5 | 0.29 +- 0.025 | 0.25 +- 0.022 |
+----------+---------------+---------------+


In [38]:
val_simple(0.80)

Validación Simple con 80% Train

+----------+---------------+---------------+
|   N_REPS | TTT_DB        | GER_DB        |
+==========+===============+===============+
|        1 | 0.26 +- 0.000 | 0.24 +- 0.000 |
+----------+---------------+---------------+
|        2 | 0.29 +- 0.016 | 0.27 +- 0.015 |
+----------+---------------+---------------+
|        3 | 0.31 +- 0.026 | 0.27 +- 0.008 |
+----------+---------------+---------------+
|        4 | 0.32 +- 0.026 | 0.26 +- 0.022 |
+----------+---------------+---------------+
|        5 | 0.29 +- 0.032 | 0.28 +- 0.053 |
+----------+---------------+---------------+


### 1.3.2 Validación cruzada

In [39]:
max_folds = 11
j = 0
L = []
for i in range(3, max_folds+1, 2):
    L.append([])
    strat = EstrategiaParticionado.ValidacionCruzada(k_fold=i)
    errores_ttt = NB.validacion(strat, dataset_ttt, NB)
    errores_ger = NB.validacion(strat, dataset_ger, NB)
    L[j].append(str(i))
    L[j].append("%.2f +- %.3f"% (np.mean(errores_ttt), np.std(errores_ttt)))
    L[j].append("%.2f +- %.3f"% (np.mean(errores_ger), np.std(errores_ger)))
    j+=1
print(tabulate(L, headers = ["K_FOLDS", "TTT_DB", "GER_DB"], tablefmt="grid"))

+-----------+---------------+---------------+
|   K_FOLDS | TTT_DB        | GER_DB        |
+===========+===============+===============+
|         3 | 0.30 +- 0.030 | 0.26 +- 0.005 |
+-----------+---------------+---------------+
|         5 | 0.30 +- 0.026 | 0.25 +- 0.017 |
+-----------+---------------+---------------+
|         7 | 0.29 +- 0.021 | 0.25 +- 0.045 |
+-----------+---------------+---------------+
|         9 | 0.30 +- 0.037 | 0.26 +- 0.038 |
+-----------+---------------+---------------+
|        11 | 0.31 +- 0.056 | 0.25 +- 0.032 |
+-----------+---------------+---------------+


# <font color='red'>2. Entrenamiento y clasificación con funciones de sklearn (encapsuladas en Clasificador.py)</font>

In [40]:
# Las funciones de sklearn requieren una codificación de los atributos discretos diferente a la nuestra.
# Por lo tanto, realizamos un pequeño trabajo de preprocesado de datos para que sus funciones de sklearn
# puiedan realizar un entrenamiento y clasificación equiparable al nuestro

def preprocess_data(dataset):
    X = dataset.datos[:,:-1] # all rows, all columns but last one
    Y = dataset.datos[:,-1]  # all rows, just last column (class)
    
    # One hot encoding for discrete features
    enc = OneHotEncoder(sparse=False, categories='auto')
    X_enc = np.array(enc.fit_transform(X)) 
    # Concatenating encoded data matrix and classes
    datos_aux = np.concatenate((X_enc, Y[:,None]), axis=1)
    # Updating dataset
    dataset.datos = np.zeros(datos_aux.shape)
    dataset.datos = datos_aux
    return dataset 

## <font color='blue'> 2.1 Dataset tic-tac-toe </font>

In [41]:
ttt_db = Datos('../ConjuntosDatos/tic-tac-toe.data')
ndata, ncols = ttt_db.datos.shape # nfeat = ncols-1 
print("Numero de ejemplos de entrenamiento:", ndata, ", Numero de columnas:", ncols)

Numero de ejemplos de entrenamiento: 958 , Numero de columnas: 10


In [42]:
ttt_db = preprocess_data(ttt_db)
np.random.shuffle(ttt_db.datos)

NBSK = Clasificador.ClasificadorNaiveBayesSK(gaussian_feat=False)

### 2.1.1 Validación simple

In [43]:
strat_simple_sk = EstrategiaParticionado.ValidacionSimple(0.8)

errs = NBSK.validacion(strat_simple_sk, ttt_db, NBSK)

print(errs)

[0.30208333333333337]


### 2.1.2 Validación simple con varias repeticiones (3 reps)

In [44]:
strat_simple_rep3_sk = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)

errs = NBSK.validacion(strat_simple_rep3_sk, ttt_db, NBSK)

print("Errores: ", errs)
print("Error medio:", np.mean(errs), "+-", np.std(errs))

Errores:  [0.38020833333333337, 0.27604166666666663, 0.33854166666666663]
Error medio: 0.3315972222222222 +- 0.04280843057617347


### 2.1.3 Validacion cruzada (5 folds)

In [45]:
strat_cross_sk = EstrategiaParticionado.ValidacionCruzada(k_fold=5)

errs = NBSK.validacion(strat_cross_sk, ttt_db, NBSK)

print("Errores: ", errs)
print("Error medio:", np.mean(errs), "+-", np.std(errs))

Errores:  [0.33507853403141363, 0.3089005235602095, 0.2984293193717278, 0.2774869109947644, 0.25130890052356025]
Error medio: 0.29424083769633513 +- 0.028369041223338356


## <font color='blue'> 2.2 Dataset German (todos los atributos se consideran discretos)</font>

In [46]:
ger_db = Datos('../ConjuntosDatos/german.data')
ndata, ncols = ger_db.datos.shape # nfeat = ncols-1 
print("Numero de ejemplos de entrenamiento:", ndata, ", Numero de columnas:", ncols)

Numero de ejemplos de entrenamiento: 1000 , Numero de columnas: 21


In [47]:
ger_db = preprocess_data(ger_db)
np.random.shuffle(ger_db.datos)

NBSK = Clasificador.ClasificadorNaiveBayesSK(gaussian_feat=False)

### 2.2.1 Validación simple

In [48]:
strat_simple_sk = EstrategiaParticionado.ValidacionSimple(0.8)

errs = NBSK.validacion(strat_simple_sk, ger_db, NBSK)

print(errs)

[0.31499999999999995]


### 2.2.2 Validación simple con varias repeticiones (3 reps)

In [49]:
strat_simple_rep3_sk = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)

errs = NBSK.validacion(strat_simple_rep3_sk, ger_db, NBSK)

print("Errores: ", errs)
print("Error medio:", np.mean(errs), "+-", np.std(errs))

Errores:  [0.28, 0.24, 0.29000000000000004]
Error medio: 0.27 +- 0.021602468994692887


### 2.2.3 Validación cruzada (5 folds)

In [50]:
strat_cross_sk = EstrategiaParticionado.ValidacionCruzada(k_fold=5)

errs = NBSK.validacion(strat_cross_sk, ger_db, NBSK)

print("Errores: ", errs)
print("Error medio:", np.mean(errs), "+-", np.std(errs))

Errores:  [0.28, 0.30000000000000004, 0.28, 0.255, 0.255]
Error medio: 0.274 +- 0.017146428199482262


## <font color='blue'> 2.3 Dataset German (todos los atributos se consideran continuos)</font>

In [51]:
ger_db_cont = Datos('../ConjuntosDatos/german.data')
ndata, ncols = ger_db_cont.datos.shape # nfeat = ncols-1 
print("Numero de ejemplos de entrenamiento:", ndata, ", Numero de columnas:", ncols)

Numero de ejemplos de entrenamiento: 1000 , Numero de columnas: 21


In [52]:
np.random.shuffle(ger_db.datos)

NBSK = Clasificador.ClasificadorNaiveBayesSK(gaussian_feat=True)

### 2.3.1 Validación simple

In [53]:
strat_simple_sk = EstrategiaParticionado.ValidacionSimple(0.8)

errs = NBSK.validacion(strat_simple_sk, ger_db_cont, NBSK)

print(errs)

[0.25]


### 2.3.2 Validación simple con varias repeticiones (3 reps)

In [54]:
strat_simple_rep3_sk = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)

errs = NBSK.validacion(strat_simple_rep3_sk, ger_db_cont, NBSK)

print("Errores: ", errs)
print("Error medio:", np.mean(errs), "+-", np.std(errs))

Errores:  [0.265, 0.25, 0.30500000000000005]
Error medio: 0.2733333333333334 +- 0.023213980461973552


### 2.3.3 Validación cruzada (5 folds)

In [55]:
strat_cross_sk = EstrategiaParticionado.ValidacionCruzada(k_fold=5)

errs = NBSK.validacion(strat_cross_sk, ger_db_cont, NBSK)

print("Errores: ", errs)
print("Error medio:", np.mean(errs), "+-", np.std(errs))

Errores:  [0.28, 0.265, 0.245, 0.28, 0.26]
Error medio: 0.266 +- 0.01319090595827293


## <font color='green'> 2.4 Tablas de Resultados </font>

### 2.4.1 Validación simple con número variable de repeticiones

In [64]:
def val_simpleSK(percentage):
    dataset_ttt = Datos("../ConjuntosDatos/tic-tac-toe.data")
    dataset_ger_d = Datos("../ConjuntosDatos/german.data")    # Dataset para el caso Discreto
    dataset_ger_c = Datos("../ConjuntosDatos/german.data")    # Dataset para el caso Continuo
    np.random.shuffle(dataset_ttt.datos)
    np.random.shuffle(dataset_ger_d.datos)
    np.random.shuffle(dataset_ger_c.datos)
    NBSK = Clasificador.ClasificadorNaiveBayesSK(gaussian_feat=False) # Clasificador Para TTT
    NBSK_d = Clasificador.ClasificadorNaiveBayesSK(gaussian_feat=False) # Clasificador Para GER en caso Discreto
    NBSK_c = Clasificador.ClasificadorNaiveBayesSK(gaussian_feat=True) # Clasificador Para GER en caso Continuo
    max_reps = 5
    L = []
    for i in range(1, max_reps+1):
        L.append([])
        strat = EstrategiaParticionado.ValidacionSimple(percentage, nreps=i)
        errores_ttt = NBSK.validacion(strat, dataset_ttt, NBSK)
        errores_ger_d = NBSK_d.validacion(strat, preprocess_data(dataset_ger_d), NBSK_d)
        errores_ger_c = NBSK_c.validacion(strat, dataset_ger_c, NBSK_c)
        L[i-1].append(str(i))
        L[i-1].append("%.2f +- %.3f"% (np.mean(errores_ttt), np.std(errores_ttt)))
        L[i-1].append("%.2f +- %.3f"% (np.mean(errores_ger_d), np.std(errores_ger_d)))
        L[i-1].append("%.2f +- %.3f"% (np.mean(errores_ger_c), np.std(errores_ger_c)))
    print("Validación Simple con " + str(int(percentage*100)) +   "% Train\n")
    print(tabulate(L, headers = ["N_REPS", "TTT_DB", "GER_DB_D", "GER_DB_C"], tablefmt="grid")) 

In [65]:
val_simpleSK(0.7)

Validación Simple con 70% Train

+----------+---------------+---------------+---------------+
|   N_REPS | TTT_DB        | GER_DB_D      | GER_DB_C      |
+==========+===============+===============+===============+
|        1 | 0.34 +- 0.000 | 0.27 +- 0.000 | 0.22 +- 0.000 |
+----------+---------------+---------------+---------------+
|        2 | 0.33 +- 0.030 | 0.29 +- 0.005 | 0.30 +- 0.010 |
+----------+---------------+---------------+---------------+
|        3 | 0.34 +- 0.022 | 0.27 +- 0.010 | 0.25 +- 0.020 |
+----------+---------------+---------------+---------------+
|        4 | 0.38 +- 0.019 | 0.25 +- 0.031 | 0.27 +- 0.020 |
+----------+---------------+---------------+---------------+
|        5 | 0.33 +- 0.016 | 0.26 +- 0.013 | 0.27 +- 0.030 |
+----------+---------------+---------------+---------------+


In [66]:
val_simpleSK(0.75)

Validación Simple con 75% Train

+----------+---------------+---------------+---------------+
|   N_REPS | TTT_DB        | GER_DB_D      | GER_DB_C      |
+==========+===============+===============+===============+
|        1 | 0.34 +- 0.000 | 0.26 +- 0.000 | 0.25 +- 0.000 |
+----------+---------------+---------------+---------------+
|        2 | 0.36 +- 0.013 | 0.28 +- 0.020 | 0.29 +- 0.006 |
+----------+---------------+---------------+---------------+
|        3 | 0.33 +- 0.020 | 0.25 +- 0.006 | 0.27 +- 0.021 |
+----------+---------------+---------------+---------------+
|        4 | 0.35 +- 0.027 | 0.24 +- 0.029 | 0.25 +- 0.022 |
+----------+---------------+---------------+---------------+
|        5 | 0.34 +- 0.023 | 0.28 +- 0.006 | 0.27 +- 0.023 |
+----------+---------------+---------------+---------------+


In [67]:
val_simpleSK(0.80)

Validación Simple con 80% Train

+----------+---------------+---------------+---------------+
|   N_REPS | TTT_DB        | GER_DB_D      | GER_DB_C      |
+==========+===============+===============+===============+
|        1 | 0.40 +- 0.000 | 0.25 +- 0.000 | 0.27 +- 0.000 |
+----------+---------------+---------------+---------------+
|        2 | 0.34 +- 0.010 | 0.24 +- 0.010 | 0.29 +- 0.013 |
+----------+---------------+---------------+---------------+
|        3 | 0.33 +- 0.018 | 0.29 +- 0.012 | 0.28 +- 0.009 |
+----------+---------------+---------------+---------------+
|        4 | 0.36 +- 0.028 | 0.26 +- 0.011 | 0.27 +- 0.022 |
+----------+---------------+---------------+---------------+
|        5 | 0.34 +- 0.017 | 0.27 +- 0.019 | 0.29 +- 0.021 |
+----------+---------------+---------------+---------------+


### 2.4.2 Validación cruzada </font>

In [69]:
dataset_ger_d = Datos("../ConjuntosDatos/german.data")    # Dataset para el caso Discreto
dataset_ger_c = Datos("../ConjuntosDatos/german.data")    # Dataset para el caso Continuo
np.random.shuffle(dataset_ger_d.datos)
np.random.shuffle(dataset_ger_c.datos)

preprocess_data(dataset_ger_d)

NBSK = Clasificador.ClasificadorNaiveBayesSK(gaussian_feat=False) # Clasificador Para TTT
NBSK_d = Clasificador.ClasificadorNaiveBayesSK(gaussian_feat=False) # Clasificador Para GER en caso Discreto
NBSK_c = Clasificador.ClasificadorNaiveBayesSK(gaussian_feat=True) # Clasificador Para GER en caso Continuo

max_folds = 11
j = 0
L = []
for i in range(3, max_folds+1, 2):
    L.append([])
    strat = EstrategiaParticionado.ValidacionCruzada(k_fold=i)
    errores_ttt = NBSK.validacion(strat, dataset_ttt, NBSK)
    errores_ger_d = NBSK_d.validacion(strat, dataset_ger_d, NBSK_d)
    errores_ger_c = NBSK_c.validacion(strat, dataset_ger_c, NBSK_c)
    L[j].append(str(i))
    L[j].append("%.2f +- %.3f"% (np.mean(errores_ttt), np.std(errores_ttt)))
    L[j].append("%.2f +- %.3f"% (np.mean(errores_ger_d), np.std(errores_ger_d)))
    L[j].append("%.2f +- %.3f"% (np.mean(errores_ger_c), np.std(errores_ger_c)))
    j+=1
print(tabulate(L, headers = ["K_FOLDS", "TTT_DB", "GER_DB_D", "GER_DB_C"], tablefmt="grid")) 

+-----------+---------------+---------------+---------------+
|   K_FOLDS | TTT_DB        | GER_DB_D      | GER_DB_C      |
+===========+===============+===============+===============+
|         3 | 0.34 +- 0.017 | 0.27 +- 0.015 | 0.26 +- 0.016 |
+-----------+---------------+---------------+---------------+
|         5 | 0.34 +- 0.028 | 0.27 +- 0.041 | 0.27 +- 0.032 |
+-----------+---------------+---------------+---------------+
|         7 | 0.35 +- 0.022 | 0.27 +- 0.031 | 0.26 +- 0.035 |
+-----------+---------------+---------------+---------------+
|         9 | 0.34 +- 0.033 | 0.27 +- 0.045 | 0.26 +- 0.031 |
+-----------+---------------+---------------+---------------+
|        11 | 0.34 +- 0.042 | 0.25 +- 0.051 | 0.27 +- 0.049 |
+-----------+---------------+---------------+---------------+


# <font color='red'>3. Análisis ROC </font>

In [73]:
def create_confMatrix(pred, real):
    i = 0
    tp, fp, tn, fn = 0, 0, 0, 0
    L = [[],[]]
    
    for pred_val in pred:
        # True negative
        if pred_val==0 and pred_val==real[i]:
            tn += 1
        # False negative
        elif pred_val==0 and pred_val!=real[i]:
            fn += 1
        # True positive
        elif pred_val==1 and pred_val==real[i]:
            tp += 1
        # False positive
        elif pred_val==1 and pred_val!=real[i]:
            fp += 1
        i += 1
        
    L[0].append(tp)
    L[0].append(fn)
    L[1].append(fp)
    L[1].append(tn) 
    
    return L



def print_confMatrix(m):
    print(tabulate([["", "POS", "NEG"], ["POS", m[0][0], m[0][1]], ["NEG", m[1][0], m[1][1]]],
                   headers="firstrow", tablefmt="grid")) 

In [74]:
def ROC_Analysis(dataset, clf, particionado):
    seed = random.seed(0)
    np.random.shuffle(dataset.datos)
    particionado.creaParticiones(dataset.datos, seed)
    pred = []
    real = []
    for particion in particionado.particiones:

        datostrain = dataset.datos[particion.indicesTrain, :]
        datostest = dataset.datos[particion.indicesTest, :]
            
        clf.entrenamiento(datostrain, dataset.nominalAtributos, dataset.diccionario)
            
        pred = clf.clasifica(datostest, dataset.nominalAtributos, dataset.diccionario)
        real = datostest[:,-1] 
        
        m = create_confMatrix(pred, real)
        rtp = int(m[0][0]) / (int(m[0][0]) + int(m[0][1]))
        rtn = int(m[1][1]) / (int(m[1][1]) + int(m[1][0]))
        
    print_confMatrix(m)
    print(rtp, rtn)
 

In [93]:
dataset_ttt = Datos("../ConjuntosDatos/tic-tac-toe.data")
dataset_ger = Datos("../ConjuntosDatos/german.data")
np.random.shuffle(dataset_ttt.datos)
np.random.shuffle(dataset_ger.datos)
particionado = EstrategiaParticionado.ValidacionSimple(0.8)

clf = Clasificador.ClasificadorNaiveBayesSK(gaussian_feat=True)

ROC_Analysis(dataset_ger, clf, particionado)

+-----+-------+-------+
|     |   POS |   NEG |
+=====+=======+=======+
| POS |    34 |    17 |
+-----+-------+-------+
| NEG |    31 |   118 |
+-----+-------+-------+
0.6666666666666666 0.7919463087248322
